In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
print(tf.__version__)

2.2.0-rc1


In [0]:
from tensorflow.keras.layers import Dense, Input, Embedding, Concatenate, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [3]:
!wget -nc http://files.grouplens.org/datasets/movielens/ml-20m.zip

--2020-03-28 13:02:39--  http://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  91.6MB/s    in 2.1s    

2020-03-28 13:02:42 (91.6 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]



In [4]:
!unzip -n ml-20m.zip

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         


In [6]:
!ls

ml-20m	ml-20m.zip  sample_data


In [7]:
df = pd.read_csv('ml-20m/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [0]:
df.userId = pd.Categorical(df.userId)
df['new_user_id'] = df.userId.cat.codes

In [0]:
df.movieId = pd.Categorical(df.movieId)
df['new_movie_id'] = df.movieId.cat.codes

In [0]:
user_ids = df['new_user_id'].values
movie_ids = df['new_movie_id'].values
ratings = df['rating'].values

In [0]:
N = len(set(user_ids))
M = len(set(movie_ids))
K = 10

In [0]:
u = Input(shape=(1,))
m = Input(shape=(1,))
u_emb = Embedding(N, K)(u)
m_emb = Embedding(M, K)(m)
u_emb = Flatten()(u_emb)
m_emb = Flatten()(m_emb)
x = Concatenate()([u_emb, m_emb])
x = Dense(1024, activation='relu')(x)
x = Dense(1)(x)



In [19]:
model = Model(inputs=[u, m], outputs=x)
model.compile(
    loss='mse',
    optimizer=SGD(lr=0.08, momentum=0.9)
)


ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [0]:
user_ids, movie_ids, ratings = shuffle(user_ids, movie_ids, ratings)
Ntrain = int(0.8*len(ratings))
train_user = user_ids[:Ntrain]
train_movie = movie_ids[:Ntrain]
train_ratings = ratings[:Ntrain]
test_user = user_ids[Ntrain:]
test_movie = movie_ids[Ntrain:]
test_ratings = ratings[Ntrain:]

avg_rating = train_ratings.mean()
train_ratings = train_ratings - avg_rating
test_ratings = test_ratings - avg_rating


In [21]:
r = model.fit(
    x=[train_user, train_movie],
    y = train_ratings,
    epochs=25,
    batch_size=1024,
    verbose=2,
    validation_data=([test_user,test_movie], test_ratings)
)

Epoch 1/25
15626/15626 - 58s - loss: 0.7757 - val_loss: 0.7209
Epoch 2/25
15626/15626 - 57s - loss: 0.7012 - val_loss: 0.6979
Epoch 3/25
15626/15626 - 57s - loss: 0.6801 - val_loss: 0.6833
Epoch 4/25
15626/15626 - 57s - loss: 0.6643 - val_loss: 0.6764
Epoch 5/25
15626/15626 - 57s - loss: 0.6537 - val_loss: 0.6717
Epoch 6/25
15626/15626 - 58s - loss: 0.6424 - val_loss: 0.6620
Epoch 7/25
15626/15626 - 57s - loss: 0.6279 - val_loss: 0.6496
Epoch 8/25
15626/15626 - 56s - loss: 0.6122 - val_loss: 0.6422
Epoch 9/25
15626/15626 - 57s - loss: 0.6017 - val_loss: 0.6401
Epoch 10/25
15626/15626 - 56s - loss: 0.5934 - val_loss: 0.6361
Epoch 11/25
15626/15626 - 57s - loss: 0.5856 - val_loss: 0.6342
Epoch 12/25
15626/15626 - 56s - loss: 0.5781 - val_loss: 0.6329
Epoch 13/25
15626/15626 - 56s - loss: 0.5709 - val_loss: 0.6298
Epoch 14/25
15626/15626 - 56s - loss: 0.5643 - val_loss: 0.6328
Epoch 15/25
15626/15626 - 54s - loss: 0.5587 - val_loss: 0.6276
Epoch 16/25
15626/15626 - 54s - loss: 0.5541 - va